In [ ]:

!python3 -m pip install tensorflow[and-cuda]==2.19


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.3/363.3 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 895.7/895.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━ 327.1/577.2 MB 1.5 MB/s eta 0:02:53
ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/usr/local/lib/python3.12/dist-packages/pip/_vendor/urllib3/response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_vendor/urllib3/response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-pa

In [ ]:
!pip install scipy

In [ ]:
!pip install wfdb
!wget -r -N -c -np https://physionet.org/files/sddb/1.0.0/
!which python

In [ ]:
!pip install wfdb
import numpy as np
import tensorflow as tf
from scipy.signal import butter, filtfilt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout, MultiHeadAttention, LayerNormalization, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import wfdb
import pandas as pd

In [ ]:
fs = 250  # Sampling frequency
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut=0.5, highcut=100, fs=250, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    return filtfilt(b, a, data)

def load_sddb_lead_ii(file_number):
    record = wfdb.rdrecord(f'/content/physionet.org/files/sddb/1.0.0/{file_number}')
    signal = record.p_signal[:, 1]
    filtered = bandpass_filter(signal)
    normalized = (filtered - np.mean(filtered)) / np.std(filtered)
    return normalized
ecg_signals = []
for file_num in range(30, 50):
    try:
        ecg = load_sddb_lead_ii(file_num)
        ecg_signals.append(ecg)
        print("loaded file")
    except Exception as e:
        print(f"Skipping file {file_num}: {e}")

for file_num in range(52, 53):
    try:
        ecg = load_sddb_lead_ii(file_num)
        ecg_signals.append(ecg)
        print("loaded file")
    except Exception as e:
        print(f"Skipping file {file_num}: {e}")

loaded file
loaded file
loaded file
loaded file
loaded file
loaded file
loaded file
loaded file
loaded file
loaded file
loaded file
loaded file
loaded file
loaded file
loaded file
loaded file
loaded file
loaded file
loaded file
loaded file
loaded file


data iwndow


In [ ]:
vf_onset_times_in_samples = {
    30: 28473,   # 07:54:33
    31: 49344,   # 13:42:24
    32: 60318,   # 16:45:18
    33: 17179,   # 04:46:19
    34: 23744,   # 06:35:44
    35: 88496,   # 24:34:56
    36: 68341,   # 18:59:01
    37: 5473,    # 01:31:13
    38: 28914,   # 08:01:54   (VF onset present)
    39: 16671,   # 04:37:51
    40: None,    # paced, no VF
    41: 10764,   # 02:59:24
    42: None,    # no VF
    43: 56231,   # 15:37:11
    44: 70725,   # 19:38:45
    45: 65357,   # 18:09:17
    46: 13307,   # 03:41:47
    47: 22381,   # 06:13:01
    48: 8980,    # 02:29:40
    49: None,    # paced, no VF
    50: 42343,   # 11:45:43
    51: 82703,   # 22:58:23
    52: 9160     # 02:32:40
}

print("VF onset times in samples dictionary created.")

VF onset times in samples dictionary created.


In [ ]:
window_size = fs * 60  # 5 minutes
stride = fs * 150  # 50% overlap

def get_vf_onset_time(file_number):
  vf_onset_sample = vf_onset_times_in_samples.get(file_number)
  if vf_onset_sample is not None:
      print(f"Retrieved VF onset sample for file {file_number}: {vf_onset_sample}")
  else:
      print(f"No VF onset sample found for file {file_number}, or it is None.")
  return vf_onset_sample

X, y = [], []

for file_num, signal in zip(range(30, 53), ecg_signals):
    vf_sample = get_vf_onset_time(file_num)

    if len(signal) < window_size:
        print(f"Skipping file {file_num} due to insufficient signal length.")
        continue

    for start in range(0, len(signal) - window_size + 1, stride):
        window = signal[start:start + window_size]
        print(f"start time {start}")
        # Ensure window has correct size, if not, skip
        if len(window) != window_size:
            print(f"Skipping window due to incorrect size: {len(window)} instead of {window_size}")
            continue

        window_center = start + window_size // 2
        print(f"window center {window_center}")
        if vf_sample is not None:
          time_diff_sec = vf_sample
        else:
          time_diff_sec = None
        if time_diff_sec == None:
          risk_class = 0
        elif time_diff_sec < 0:
            continue  # after VF event, ignore
        elif time_diff_sec > 24 * 3600:
            risk_class = 0  # low risk (>24 hrs before VF)
        elif time_diff_sec > 12 * 3600:
            risk_class = 1  # medium risk (12-24 hrs)
        elif time_diff_sec > 6 * 3600:
            risk_class = 2  # high risk (6-12 hrs)
        else:
            risk_class = 3  # critical risk (<6 hrs)

        X.append(window)
        y.append(risk_class)
print(y)
X = np.array(X).reshape(-1, window_size, 1)  # Add channel dimension
y = np.array(y)

[]


NameError: name 'np' is not defined

**Reasoning**:
The `X` and `y` arrays have been regenerated with the corrected VF onset times and updated risk class assignments. The next step is to re-apply the data augmentation to these new `X` and `y` arrays, as done previously in the notebook, to ensure the training data is appropriately augmented before splitting.



In [ ]:
def augment_ecg(ecg):
    shift = np.random.randint(-fs*5, fs*5)
    ecg = np.roll(ecg, shift, axis=0)

    scale = np.random.uniform(0.95, 1.05)
    ecg = ecg * scale

    noise = np.random.normal(0, 0.01, size=ecg.shape)
    ecg = ecg + noise

    return ecg

X_aug, y_aug = [], []

for i in range(len(X)):
    for _ in range(2):
        X_aug.append(augment_ecg(np.copy(X[i])))
        y_aug.append(y[i])

X = np.concatenate([X, np.array(X_aug)], axis=0)
y = np.concatenate([y, np.array(y_aug)], axis=0)


In [ ]:
from sklearn.model_selection import train_test_split

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.05, stratify=y, random_state=42)
val_ratio = 0.15 / 0.95
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=val_ratio, stratify=y_temp, random_state=42)

print(f"Train size: {X_train.shape[0]}, Validation size: {X_val.shape[0]}, Test size: {X_test.shape[0]}")

In [ ]:
X_train_flat = X_train.reshape(X_train.shape[0], -1)

valid_mask = (~np.any(np.isnan(X_train_flat), axis=1)) & (np.any(X_train_flat != 0, axis=1))

X_train_valid = X_train_flat[valid_mask]
y_train_valid = y_train[valid_mask]

X_train_invalid = X_train_flat[~valid_mask]
y_train_invalid = y_train[~valid_mask]

# Check class distribution in valid data
import numpy as np
unique, counts = np.unique(y_train_valid, return_counts=True)
print(f'Classes in SMOTE-valid data: {dict(zip(unique, counts))}')

# Apply SMOTE on only valid subset if >1 class present
from imblearn.over_sampling import SMOTE
if len(unique) > 1:
    smote = SMOTE(random_state=42)
    X_res_valid, y_res_valid = smote.fit_resample(X_train_valid, y_train_valid)
else:
    print("Warning: Only one class in SMOTE-valid data. Skipping SMOTE.")
    X_res_valid, y_res_valid = X_train_valid, y_train_valid

X_res_valid = X_res_valid.reshape(-1, window_size, 1)

X_train_clean = np.concatenate([X_res_valid, X_train_invalid.reshape(-1, window_size, 1)], axis=0)
y_train_clean = np.concatenate([y_res_valid, y_train_invalid], axis=0)

print(f'Final training set after SMOTE and reintegrating invalid windows: {X_train_clean.shape[0]} samples')

import pandas as pd
print("\nClass distribution in y_train_clean:")
print(pd.Series(y_train_clean).value_counts())

print("\nClass distribution in y_val:")
print(pd.Series(y_val).value_counts())

print("\nClass distribution in y_test:")
print(pd.Series(y_test).value_counts())

In [ ]:
time_diffs = []

for file_num, signal in zip(range(30, 53), ecg_signals):
    vf_sample = vf_onset_times_in_samples.get(file_num)

    # Only consider signals that are long enough for at least one window
    if len(signal) < window_size:
        continue

    for start in range(0, len(signal) - window_size + 1, stride):
        window_center = start + window_size // 2
        if vf_sample == None:
          vf_sample = 0
        time_diffs.append(vf_sample)

plt.figure(figsize=(12, 6))
plt.hist(time_diffs, bins=50, edgecolor='black')
plt.title('Distribution of Time Differences (seconds) from VF Onset (with updated window size)')
plt.xlabel('Time Difference to VF Onset (seconds)')
plt.ylabel('Number of Windows')

# Original risk thresholds for reference
plt.axvline(x=6 * 3600, color='r', linestyle=':', label='Original 6 hours (21600s) threshold')
plt.axvline(x=12 * 3600, color='g', linestyle=':', label='Original 12 hours (43200s) threshold')
plt.axvline(x=24 * 3600, color='b', linestyle=':', label='Original 24 hours (86400s) threshold')

# Current risk thresholds for reference (from cell 5d84e860)
plt.axvline(x=6 * 3600, color='r', linestyle='--', label='Current 6 hours (21600s) threshold')
plt.axvline(x=12 * 3600, color='g', linestyle='--', label='Current 12 hours (43200s) threshold')
plt.axvline(x=24 * 3600, color='b', linestyle='--', label='Current 24 hours (86400s) threshold')

plt.legend()
plt.show()

print(f"Minimum time difference: {np.min(time_diffs)} seconds")
print(f"Maximum time difference: {np.max(time_diffs)} seconds")
print(f"Mean time difference: {np.mean(time_diffs)} seconds")
print(f"Median time difference: {np.median(time_diffs)} seconds")

In [ ]:
window_size = X_train_clean.shape[1]
inputs = Input(shape=(window_size, 1))

x = Conv1D(64, 5, activation='relu', padding='same')(inputs)
x = MaxPooling1D(pool_size=4)(x)
x = Conv1D(128, 5, activation='relu', padding='same')(x)
x = MaxPooling1D(pool_size=4)(x)

x = Bidirectional(LSTM(64, return_sequences=True))(x)
x = Bidirectional(LSTM(32, return_sequences=True))(x)

attn_output = MultiHeadAttention(num_heads=4, key_dim=32)(x, x)
x = LayerNormalization()(attn_output)

x = GlobalAveragePooling1D()(x)
x = Dropout(0.3)(x)
outputs = Dense(4, activation='softmax')(x)

model = Model(inputs, outputs)
model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

y_train_cat = to_categorical(y_train_clean, 4)
y_val_cat = to_categorical(y_val, 4)
y_test_cat = to_categorical(y_test, 4)

callbacks = [
    EarlyStopping(patience=1000, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.5, patience=5),
    ModelCheckpoint('sddb_single_lead_best_model.h5', save_best_only=True, monitor='val_loss')
]

history = model.fit(
    X_train_clean, y_train_cat,
    validation_data=(X_val, y_val_cat),
    epochs=500,
    batch_size=32,
    callbacks=callbacks
)

# Task
The model shows 'nan' loss, which indicates a problem with the data. I'll modify the `load_sddb_lead_ii` function to handle cases where the signal's standard deviation is zero by returning `None` for such invalid signals. I'll also update the loops that populate `ecg_signals` to store valid signals in a dictionary, preventing NaNs from entering the dataset. Finally, I will re-run the subsequent data processing and model training steps.

I'll start by modifying the `load_sddb_lead_ii` function in cell `w7zum4zNTnr8` to properly handle potential NaN values by checking the standard deviation before normalization.

## Refine Data Loading and NaN Handling

### Subtask:
Modify the `load_sddb_lead_ii` function in cell `w7zum4zNTnr8` to handle cases where the signal's standard deviation is zero after filtering. This function will now return `None` for such invalid signals. Update the loops that populate `ecg_signals` to store valid signals in a dictionary, `ecg_signals_dict`, mapping `file_num` to the signal, ensuring proper alignment and skipping of invalid signals.


**Reasoning**:
The subtask requires modifying the `load_sddb_lead_ii` function to handle zero standard deviation and changing how ECG signals are stored from a list to a dictionary, skipping invalid signals. This change must be applied to the existing code cell `w7zum4zNTnr8`.



In [ ]:
fs = 250  # Sampling frequency
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut=0.5, highcut=100, fs=250, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    return filtfilt(b, a, data)

def load_sddb_lead_ii(file_number):
    record = wfdb.rdrecord(f'/content/dataset/{file_number}')
    signal = record.p_signal[:, 1]  # Lead II is channel index 1
    filtered = bandpass_filter(signal)

    # Add check for zero standard deviation
    if np.std(filtered) == 0:
        print(f"Warning: Standard deviation is zero for file {file_number}. Returning None.")
        return None

    normalized = (filtered - np.mean(filtered)) / np.std(filtered)
    return normalized

ecq_signals_dict = {}
for file_num in range(30, 50):
    try:
        ecg = load_sddb_lead_ii(file_num)
        if ecg is not None:
            ecq_signals_dict[file_num] = ecg
            print(f"Loaded file {file_num}")
        else:
            print(f"Skipping file {file_num} due to invalid signal.")
    except Exception as e:
        print(f"Skipping file {file_num}: {e}")

for file_num in range(52, 53):
    try:
        ecg = load_sddb_lead_ii(file_num)
        if ecg is not None:
            ecq_signals_dict[file_num] = ecg
            print(f"Loaded file {file_num}")
        else:
            print(f"Skipping file {file_num} due to invalid signal.")
    except Exception as e:
        print(f"Skipping file {file_num}: {e}")

**Reasoning**:
The subtask requires modifying the `load_sddb_lead_ii` function to handle zero standard deviation and changing how ECG signals are stored from a list to a dictionary, skipping invalid signals. This change must be applied to the existing code cell `w7zum4zNTnr8`.



In [ ]:
fs = 250  # Sampling frequency
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut=0.5, highcut=100, fs=250, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    return filtfilt(b, a, data)

def load_sddb_lead_ii(file_number):
    record = wfdb.rdrecord(f'/content/dataset/{file_number}')
    signal = record.p_signal[:, 1]  # Lead II is channel index 1
    filtered = bandpass_filter(signal)

    # Add check for zero standard deviation
    if np.std(filtered) == 0:
        print(f"Warning: Standard deviation is zero for file {file_number}. Returning None.")
        return None

    normalized = (filtered - np.mean(filtered)) / np.std(filtered)
    return normalized

ecq_signals_dict = {}
for file_num in range(30, 50):
    try:
        ecg = load_sddb_lead_ii(file_num)
        if ecg is not None:
            ecq_signals_dict[file_num] = ecg
            print(f"Loaded file {file_num}")
        else:
            print(f"Skipping file {file_num} due to invalid signal.")
    except Exception as e:
        print(f"Skipping file {file_num}: {e}")

for file_num in range(52, 53):
    try:
        ecg = load_sddb_lead_ii(file_num)
        if ecg is not None:
            ecq_signals_dict[file_num] = ecg
            print(f"Loaded file {file_num}")
        else:
            print(f"Skipping file {file_num} due to invalid signal.")
    except Exception as e:
        print(f"Skipping file {file_num}: {e}")

**Reasoning**:
The subtask requires modifying the `load_sddb_lead_ii` function to handle zero standard deviation and changing how ECG signals are stored from a list to a dictionary, skipping invalid signals. This change must be applied to the existing code cell `w7zum4zNTnr8`.



# **DDIOSDAHUIOASJKD ASD\\ IDHAR SE SHURU KAR**

In [ ]:
fs = 250  # Sampling frequency
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut=0.5, highcut=100, fs=250, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    return filtfilt(b, a, data)

def load_sddb_lead_ii(file_number):
    record = wfdb.rdrecord(f'/content/dataset/{file_number}')
    signal = record.p_signal[:, 1]  # Lead II is channel index 1
    filtered = bandpass_filter(signal)

    # Add check for zero standard deviation
    if np.std(filtered) == 0:
        print(f"Warning: Standard deviation is zero for file {file_number}. Returning None.")
        return None

    normalized = (filtered - np.mean(filtered)) / np.std(filtered)
    return normalized

ecq_signals_dict = {}
for file_num in range(30, 50):
    try:
        ecg = load_sddb_lead_ii(file_num)
        if ecg is not None:
            ecq_signals_dict[file_num] = ecg
            print(f"Loaded file {file_num}")
        else:
            print(f"Skipping file {file_num} due to invalid signal.")
    except Exception as e:
        print(f"Skipping file {file_num}: {e}")

for file_num in range(52, 53):
    try:
        ecg = load_sddb_lead_ii(file_num)
        if ecg is not None:
            ecq_signals_dict[file_num] = ecg
            print(f"Loaded file {file_num}")
        else:
            print(f"Skipping file {file_num} due to invalid signal.")
    except Exception as e:
        print(f"Skipping file {file_num}: {e}")

**Reasoning**:
The previous step successfully modified the `load_sddb_lead_ii` function and created `ecq_signals_dict`. Now, I need to update the data windowing and risk class assignment logic to use this new dictionary and handle `None` values correctly, regenerating the `X` and `y` arrays with valid data.



In [ ]:
window_size = fs * 60  # 5 minutes
stride = fs * 150  # 50% overlap

def get_vf_onset_time(file_number):
  # Use the pre-defined dictionary for VF onset times
  vf_onset_sample = vf_onset_times_in_samples.get(file_number)
  if vf_onset_sample is not None:
      print(f"Retrieved VF onset sample for file {file_number}: {vf_onset_sample}")
  else:
      print(f"No VF onset sample found for file {file_number}, or it is None.")
  return vf_onset_sample

X, y = [], []

for file_num, signal in ecq_signals_dict.items():
    vf_sample = get_vf_onset_time(file_num)

    # Only consider signals that are long enough for at least one window
    if len(signal) < window_size:
        print(f"Skipping file {file_num} due to insufficient signal length.")
        continue

    for start in range(0, len(signal) - window_size + 1, stride):
        window = signal[start:start + window_size]

        # Ensure window has correct size, if not, skip
        if len(window) != window_size:
            print(f"Skipping window due to incorrect size: {len(window)} instead of {window_size}")
            continue

        window_center = start + window_size // 2

        # Calculate time difference from window center to VF onset in samples
        # Convert to seconds for comparison with thresholds
        if vf_sample is not None:
            # If vf_sample exists, calculate time difference from window_center to vf_sample in seconds
            time_diff_samples = vf_sample * fs
            time_diff_sec = time_diff_samples / fs
        else:
            # If no VF onset, assign class 0 (low risk)
            time_diff_sec = None

        # Assign new risk classes based on adjusted time differences (seconds)
        if time_diff_sec is None:
            risk_class = 0  # Low risk (no VF onset)
        elif time_diff_sec < 0: # This case means the window is after the VF event
            continue  # Skip windows after the VF event
        elif time_diff_sec > 24 * 3600:
            risk_class = 0  # low risk (>24 hrs before VF)
        elif time_diff_sec > 12 * 3600:
            risk_class = 1  # medium risk (12-24 hrs)
        elif time_diff_sec > 6 * 3600:
            risk_class = 2  # high risk (6-12 hrs)
        else:
            risk_class = 3  # critical risk (<6 hrs)

        X.append(window)
        y.append(risk_class)

# Convert to numpy arrays and reshape
X = np.array(X).reshape(-1, window_size, 1)  # Add channel dimension
y = np.array(y)

print(f"Generated {len(X)} windows with {len(y)} labels.")
print("Class distribution in generated data:")
print(pd.Series(y).value_counts().sort_index())


**Reasoning**:
The data has been processed, and the `X` and `y` arrays have been regenerated. The next step is to apply the data augmentation logic to these new `X` and `y` arrays to increase the dataset size and potentially improve class representation before splitting and balancing.



In [ ]:
def augment_ecg(ecg):
    shift = np.random.randint(-fs*5, fs*5)
    ecg = np.roll(ecg, shift, axis=0)

    scale = np.random.uniform(0.95, 1.05)
    ecg = ecg * scale

    noise = np.random.normal(0, 0.01, size=ecg.shape)
    ecg = ecg + noise

    return ecg

X_aug, y_aug = [], []

for i in range(len(X)):
    for _ in range(2):
        X_aug.append(augment_ecg(np.copy(X[i])))
        y_aug.append(y[i])

X = np.concatenate([X, np.array(X_aug)], axis=0)
y = np.concatenate([y, np.array(y_aug)], axis=0)

**Reasoning**:
The data has been augmented, so the next logical step is to re-split the data into training, validation, and test sets to prepare for model training and evaluation, following the previous notebook's structure.



In [ ]:
from sklearn.model_selection import train_test_split

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.05, stratify=y, random_state=42)
val_ratio = 0.15 / 0.95
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=val_ratio, stratify=y_temp, random_state=42)

print(f"Train size: {X_train.shape[0]}, Validation size: {X_val.shape[0]}, Test size: {X_test.shape[0]}")

**Reasoning**:
The data has been re-split after augmentation. The next logical step, as per the subtask instructions, is to re-execute the SMOTE processing step to balance the classes in the training data (`X_train`, `y_train`) and then print the class distributions. This will also re-apply the NaN/zero check to ensure data integrity for model training.



In [ ]:
X_train_flat = X_train.reshape(X_train.shape[0], -1)

valid_mask = (~np.any(np.isnan(X_train_flat), axis=1)) & (np.any(X_train_flat != 0, axis=1))

X_train_valid = X_train_flat[valid_mask]
y_train_valid = y_train[valid_mask]

X_train_invalid = X_train_flat[~valid_mask]
y_train_invalid = y_train[~valid_mask]

import numpy as np
unique, counts = np.unique(y_train_valid, return_counts=True)
print(f'Classes in SMOTE-valid data: {dict(zip(unique, counts))}')

from imblearn.over_sampling import SMOTE
if len(unique) > 1:
    smote = SMOTE(random_state=42)
    X_res_valid, y_res_valid = smote.fit_resample(X_train_valid, y_train_valid)
else:
    print("Warning: Only one class in SMOTE-valid data. Skipping SMOTE.")
    X_res_valid, y_res_valid = X_train_valid, y_train_valid

X_res_valid = X_res_valid.reshape(-1, window_size, 1)

X_train_clean = np.concatenate([X_res_valid, X_train_invalid.reshape(-1, window_size, 1)], axis=0)
y_train_clean = np.concatenate([y_res_valid, y_train_invalid], axis=0)

print(f'Final training set after SMOTE and reintegrating invalid windows: {X_train_clean.shape[0]} samples')

import pandas as pd
print("\nClass distribution in y_train_clean:")
print(pd.Series(y_train_clean).value_counts())

print("\nClass distribution in y_val:")
print(pd.Series(y_val).value_counts())

print("\nClass distribution in y_test:")
print(pd.Series(y_test).value_counts())

**Reasoning**:
The data has been preprocessed, augmented, split, and re-balanced using SMOTE. The next logical step is to re-train the model with the updated and balanced dataset to see if the 'nan' loss issue has been resolved.



In [ ]:
window_size = X_train_clean.shape[1]
inputs = Input(shape=(window_size, 1))

x = Conv1D(64, 5, activation='relu', padding='same')(inputs)
x = MaxPooling1D(pool_size=4)(x)
x = Conv1D(128, 5, activation='relu', padding='same')(x)
x = MaxPooling1D(pool_size=4)(x)

x = Bidirectional(LSTM(64, return_sequences=True))(x)
x = Bidirectional(LSTM(32, return_sequences=True))(x)

attn_output = MultiHeadAttention(num_heads=4, key_dim=32)(x, x)
x = LayerNormalization()(attn_output)

x = GlobalAveragePooling1D()(x)
x = Dropout(0.3)(x)
outputs = Dense(4, activation='softmax')(x)

model = Model(inputs, outputs)
model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

y_train_cat = to_categorical(y_train_clean, 4)
y_val_cat = to_categorical(y_val, 4)
y_test_cat = to_categorical(y_test, 4)

callbacks = [
    EarlyStopping(patience=1000, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.5, patience=5),
    ModelCheckpoint('sddb_single_lead_best_model.h5', save_best_only=True, monitor='val_loss')
]

history = model.fit(
    X_train_clean, y_train_cat,
    validation_data=(X_val, y_val_cat),
    epochs=500,
    batch_size=32,
    callbacks=callbacks
)

### Debugging NaN Loss: Check for NaNs and Infs in Training Data

In [ ]:
import numpy as np

# Check X_train_clean for NaNs or Infs
if np.any(np.isnan(X_train_clean)) or np.any(np.isinf(X_train_clean)):
    print('X_train_clean contains NaN or Inf values!')
    # Optionally, print indices or count to investigate further
    nan_count = np.sum(np.isnan(X_train_clean))
    inf_count = np.sum(np.isinf(X_train_clean))
    print(f'NaN count in X_train_clean: {nan_count}')
    print(f'Inf count in X_train_clean: {inf_count}')
else:
    print('X_train_clean is clean (no NaNs or Infs).')

# Check y_train_cat for NaNs or Infs (should be one-hot encoded, so only 0s and 1s)
if np.any(np.isnan(y_train_cat)) or np.any(np.isinf(y_train_cat)):
    print('y_train_cat contains NaN or Inf values!')
else:
    print('y_train_cat is clean (no NaNs or Infs).')

# Also check if y_train_cat is empty or all zeros, which could cause issues
if y_train_cat.size == 0:
    print('y_train_cat is empty!')
elif np.all(y_train_cat == 0):
    print('y_train_cat contains all zeros!')


# Task
Develop and evaluate a Random Forest Regressor model to predict the risk associated with VF onset using the prepared ECG time series data. This will involve understanding the model's fundamental concepts, preparing the ECG data into features (X) and target (y), training the Random Forest Regressor with appropriate hyperparameters, evaluating its performance using regression metrics such as MAE, MSE, RMSE, and R-squared, making predictions on new data, and interpreting the model's insights, including feature importance.

## Understand Random Forest Regressor Basics

### Subtask:
Explain the fundamental concepts of a Random Forest Regressor, including how it combines multiple decision trees, the role of bagging (bootstrap aggregating), and how it handles feature importance and generalization to prevent overfitting.


### Understanding Random Forest Regressor Basics

A Random Forest Regressor is a powerful **ensemble learning method** that belongs to the supervised learning paradigm. It is used for both classification and regression tasks, but in this context, we focus on regression. Its strength lies in combining the predictions from multiple simple models (decision trees) to produce a more accurate and stable prediction.

1.  **Ensemble of Decision Trees**: A Random Forest Regressor operates by constructing a "forest" of many individual decision trees during the training phase. Each tree in the forest is trained independently.

2.  **Bagging (Bootstrap Aggregating)**: The foundation of Random Forest is a technique called bagging. Instead of using the entire training dataset to train every tree, bagging works as follows:
    *   **Bootstrap Sampling**: For each decision tree, a random subset of the original training data is selected with replacement. This means some data points may be sampled multiple times for a single tree, while others may not be sampled at all. This process creates diverse training sets for each tree.
    *   **Diverse Trees**: By training each tree on a slightly different subset of the data, the individual trees tend to be somewhat de-correlated, meaning they make errors on different data points.

3.  **Feature Randomness**: To further enhance diversity and reduce correlation among trees, the Random Forest introduces another layer of randomness. At each node of a decision tree, when considering a split, the algorithm does not consider all available features. Instead, it randomly selects a subset of features (e.g., `sqrt(total_features)` or `log2(total_features)`) and only considers these selected features for finding the best split at that node. This prevents strong features from dominating all trees and encourages trees to explore different predictive pathways.

4.  **Final Prediction by Averaging**: For a regression task, once all the individual decision trees are trained, to make a prediction for a new data point, the input is passed through every tree in the forest. Each tree outputs its own prediction. The final prediction of the Random Forest Regressor is then the **average** of the predictions made by all the individual trees.

5.  **Generalization and Overfitting Prevention**: This ensemble approach significantly contributes to better generalization and reduced overfitting compared to a single decision tree. A single decision tree can easily overfit the training data by learning overly complex rules. By averaging the predictions of many diverse and relatively uncorrelated trees, the Random Forest smooths out the individual trees' errors and biases. The randomness introduced by bagging and feature selection ensures that no single tree or feature dominates, leading to a more robust and generalized model.

6.  **Feature Importance**: Random Forest Regressors also provide a convenient way to estimate the **feature importance**. This is typically calculated by measuring the average reduction in impurity (e.g., Mean Squared Error for regression) across all splits where a given feature is used, averaged over all trees in the forest. Features that contribute more to reducing impurity are considered more important, helping identify influential factors in the prediction.

### Understanding Random Forest Regressor Basics

A Random Forest Regressor is a powerful **ensemble learning method** that belongs to the supervised learning paradigm. It is used for both classification and regression tasks, but in this context, we focus on regression. Its strength lies in combining the predictions from multiple simple models (decision trees) to produce a more accurate and stable prediction.

1.  **Ensemble of Decision Trees**: A Random Forest Regressor operates by constructing a "forest" of many individual decision trees during the training phase. Each tree in the forest is trained independently.

2.  **Bagging (Bootstrap Aggregating)**: The foundation of Random Forest is a technique called bagging. Instead of using the entire training dataset to train every tree, bagging works as follows:
    *   **Bootstrap Sampling**: For each decision tree, a random subset of the original training data is selected with replacement. This means some data points may be sampled multiple times for a single tree, while others may not be sampled at all. This process creates diverse training sets for each tree.
    *   **Diverse Trees**: By training each tree on a slightly different subset of the data, the individual trees tend to be somewhat de-correlated, meaning they make errors on different data points.

3.  **Feature Randomness**: To further enhance diversity and reduce correlation among trees, the Random Forest introduces another layer of randomness. At each node of a decision tree, when considering a split, the algorithm does not consider all available features. Instead, it randomly selects a subset of features (e.g., `sqrt(total_features)` or `log2(total_features)`) and only considers these selected features for finding the best split at that node. This prevents strong features from dominating all trees and encourages trees to explore different predictive pathways.

4.  **Final Prediction by Averaging**: For a regression task, once all the individual decision trees are trained, to make a prediction for a new data point, the input is passed through every tree in the forest. Each tree outputs its own prediction. The final prediction of the Random Forest Regressor is then the **average** of the predictions made by all the individual trees.

5.  **Generalization and Overfitting Prevention**: This ensemble approach significantly contributes to better generalization and reduced overfitting compared to a single decision tree. A single decision tree can easily overfit the training data by learning overly complex rules. By averaging the predictions of many diverse and relatively uncorrelated trees, the Random Forest smooths out the individual trees' errors and biases. The randomness introduced by bagging and feature selection ensures that no single tree or feature dominates, leading to a more robust and generalized model.

6.  **Feature Importance**: Random Forest Regressors also provide a convenient way to estimate the **feature importance**. This is typically calculated by measuring the average reduction in impurity (e.g., Mean Squared Error for regression) across all splits where a given feature is used, averaged over all trees in the forest. Features that contribute more to reducing impurity are considered more important, helping identify influential factors in the prediction.

## Prepare Data for Regression

### Subtask:
Prepare the ECG data by converting categorical risk labels into continuous time-to-VF-onset values and flattening the input features for Random Forest Regressor training.


**Reasoning**:
The subtask requires preparing the ECG data for regression. This involves iterating through the processed signals, generating time windows, calculating the continuous time-to-VF-onset, handling cases with no VF onset, and finally flattening the input features for regression model training and splitting the data into training and testing sets.



In [ ]:
X_reg, y_reg = [], []

for file_num, signal in ecq_signals_dict.items():
    vf_onset_sample = vf_onset_times_in_samples.get(file_num)

    if len(signal) < window_size:
        print(f"Skipping file {file_num} due to insufficient signal length for regression.")
        continue

    for start in range(0, len(signal) - window_size + 1, stride):
        window = signal[start:start + window_size]
        window_center = start + window_size // 2

        if vf_onset_sample is None:
            # If no VF onset, assign a large positive time as 'very low risk'
            time_diff_sec = 48 * 3600  # 48 hours in seconds
        else:
            # Calculate time difference from window center to VF onset in seconds
            time_diff_sec = (vf_onset_sample - window_center) / fs

            # Skip windows that are after the VF onset
            if time_diff_sec < 0:
                continue

        X_reg.append(window)
        y_reg.append(time_diff_sec)

X_reg = np.array(X_reg)
y_reg = np.array(y_reg)

# Reshape X_reg to 2D for scikit-learn regressors
X_reg_flattened = X_reg.reshape(X_reg.shape[0], -1)

# Split data into training and testing sets
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg_flattened, y_reg, test_size=0.2, random_state=42
)

print(f"X_train_reg shape: {X_train_reg.shape}")
print(f"X_test_reg shape: {X_test_reg.shape}")
print(f"y_train_reg shape: {y_train_reg.shape}")
print(f"y_test_reg shape: {y_test_reg.shape}")

## Train the Random Forest Regressor Model

### Subtask:
Train a Random Forest Regressor model using the prepared training data (`X_train_reg`, `y_train_reg`).


**Reasoning**:
The subtask requires training a Random Forest Regressor model. This involves importing the necessary class, instantiating the model with hyperparameters, and then fitting it to the prepared training data. I will include these steps in a single code block.



In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Instantiate the Random Forest Regressor model
# Using n_estimators=100 and random_state=42 for reproducibility
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

# Fit the model to the training data
rf_regressor.fit(X_train_reg, y_train_reg)

print("Random Forest Regressor model trained successfully.")

## Evaluate Model Performance

### Subtask:
Evaluate the trained Random Forest Regressor model's performance on the test data.


**Reasoning**:
The subtask requires evaluating the trained Random Forest Regressor model's performance on the test data. This involves making predictions, calculating MAE, MSE, RMSE, and R-squared, and printing the results.



In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Make predictions on the test data
y_pred_reg = rf_regressor.predict(X_test_reg)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test_reg, y_pred_reg)
mse = mean_squared_error(y_test_reg, y_pred_reg)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_reg, y_pred_reg)

# Print the evaluation metrics
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R2) Score: {r2:.2f}")